In [1]:
import pandas

In [1]:
import selenium
print(selenium.__version__)  # This should print the installed version


4.30.0


In [3]:
import re

In [4]:
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()
base_url = "https://www.kaanoon.com/answers"



current_page = 1
extracted_data = []
extracted_urls = set()  

while len(extracted_data)<2000:
    page_url = f"{base_url}?page={current_page}"
    driver.get(page_url)
    time.sleep(2)
    try:
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.answer"))
        )

        
        question_links = driver.find_elements(By.CSS_SELECTOR, "tr.answer .title a")
        new_urls = set(link.get_attribute("href") for link in question_links)

        
        unique_new_urls = new_urls - extracted_urls
        extracted_urls.update(unique_new_urls)

        print(f"✅ Found {len(unique_new_urls)} new unique questions on page {current_page}.")

        
        for url in unique_new_urls:
            try:
                driver.get(url)
                time.sleep(1)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "js-question-timeline-container"))
                )

                
                question_title = driver.find_element(By.CSS_SELECTOR, "ul.question-title h1").text.strip()
                description = driver.find_element(By.CSS_SELECTOR, "pre.description").text.strip()

               
                answers = driver.find_elements(By.CSS_SELECTOR, "div.answer.entry")
                clean_answers = []

                for ans in answers[:5]:  
                    paragraphs = ans.find_elements(By.CSS_SELECTOR, "div.text p")
                    answer_text = " ".join([p.text.strip() for p in paragraphs])

                    
                    unwanted_phrases = ["Talk to Advocate", "Available Now", "Consultations", "Answers"]
                    for phrase in unwanted_phrases:
                        answer_text = re.sub(r"\s+", " ", answer_text.split(phrase)[0].strip())

                    if answer_text:
                        clean_answers.append(answer_text)

                
                extracted_data.append({
                    "question_url": url,
                    "full_text": re.sub(r"\s+", " ", f"{question_title}. {description}"),
                    "answers": clean_answers
                })

                print(f"✅ Extracted: {url}")
                if(len(extracted_data)>=2000):
                    break
            except Exception as e:
                print(f"❌ Error processing {url}: {e}")

        
        current_page += 1

    except Exception as e:
        print(f"❌ Error on page {current_page}: {e}")
        break  


with open("answers_data_2000.json", "w", encoding="utf-8") as json_file:
    json.dump(extracted_data, json_file, indent=4, ensure_ascii=False)

print("\n✅ All unique data saved in 'answers_data.json'!")

# Close browser
driver.quit()


✅ Found 2 new unique questions on page 1.
✅ Extracted: https://www.kaanoon.com/488199/does-the-act-of-tenant-returning-the-keys-and-landlord-accepting-the-key-end-tenancy
✅ Extracted: https://www.kaanoon.com/488254/relinquishment-deed
✅ Found 6 new unique questions on page 2.
✅ Extracted: https://www.kaanoon.com/488201/securing-my-undivided-share-through-gift-deed-or-will-a-mother-s-rightful-choice
✅ Extracted: https://www.kaanoon.com/488103/divorce-after-14-years-of-marriage
✅ Extracted: https://www.kaanoon.com/488152/incorrect-relieving-letter
✅ Extracted: https://www.kaanoon.com/488118/selected-for-govt-job-will-divorce-case-cruelty-petition-affect-my-joining
✅ Extracted: https://www.kaanoon.com/488188/alteration-of-underconstruction-apartment
✅ Extracted: https://www.kaanoon.com/488107/wife-family-threatening-us
✅ Found 1 new unique questions on page 3.
✅ Extracted: https://www.kaanoon.com/356044/withdrawal-of-false-criminal-cases-498a-and-3-4-dp-act-after-getting-mutual-divorce
✅ 

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x00007FF7880FCF25+75717]
	GetHandleVerifier [0x00007FF7880FCF80+75808]
	(No symbol) [0x00007FF787EC8F9A]
	(No symbol) [0x00007FF787EC59F4]
	(No symbol) [0x00007FF787EB6789]
	(No symbol) [0x00007FF787EB84F8]
	(No symbol) [0x00007FF787EB6A96]
	(No symbol) [0x00007FF787EB6516]
	(No symbol) [0x00007FF787EB61DA]
	(No symbol) [0x00007FF787EB3E8A]
	(No symbol) [0x00007FF787EB465C]
	(No symbol) [0x00007FF787ECCF3A]
	(No symbol) [0x00007FF787F7013E]
	(No symbol) [0x00007FF787F4737A]
	(No symbol) [0x00007FF787F6F39C]
	(No symbol) [0x00007FF787F47153]
	(No symbol) [0x00007FF787F10421]
	(No symbol) [0x00007FF787F111B3]
	GetHandleVerifier [0x00007FF7883FD6FD+3223453]
	GetHandleVerifier [0x00007FF7883F7CA2+3200322]
	GetHandleVerifier [0x00007FF788415AD3+3322739]
	GetHandleVerifier [0x00007FF7881169FA+180890]
	GetHandleVerifier [0x00007FF78811E0FF+211359]
	Get

In [5]:
len(extracted_data)

967